In [265]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 


## 定义数据库连接

In [266]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

## 导入租后数据

In [267]:
# f_path_ck = "E:/myfile/租后数据/租后应收监控excel2024-07-24.xlsx"

# df_zhys = pd.read_excel(f_path_ck,sheet_name='租后应收监控')

# df_zhys.shape

In [268]:
# df_ddfq = pd.read_excel(f_path_ck,sheet_name='订单分期表')
# df_ddfq.shape

In [269]:
# df_zh = df_ddfq.merge(df_zhys,on = '订单号',how = 'left' )
# df_zh.shape

# 获取租后数据

In [270]:
sql = '''-- 租后应收监控  
SELECT  tprm.* 
,om.`status` as 订单状态值
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
,om.order_finish_date as 订单完成时间,om.order_method
,tmu.true_name ,tmu.id_card_num,tmu.mobile , tod.product_name
from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join db_digua_business.t_order_details tod on tod.id=om.id
-- where date_format(tprm.order_create_time, '%Y-%m')>='2023-09'
;
'''

df_zhys = query(sql)
df_zhys.shape

(28889, 63)

# 剔除商家数据

In [271]:
# df_zhys["年"] = df_zhys["下单月份"].str[:4]
# pd.crosstab(dfzh1["商家名称"],dfzh1["年"],margins=True).loc[["北京海鸟窝科技有限公司","深圳优优大数据科技有限公司","租着用电脑数码","苏州蚁诺宝"],:]
# 剔除商家数据
def drop_sj(df_zhys,col1,col2):
    df_zhys.drop(df_zhys[df_zhys[col1]=="深圳优优大数据科技有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="优优2店"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="小豚租（代收）"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="苏州蚁诺宝"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="租着用电脑数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="北京海鸟窝科技有限公司"].index,inplace=True)

    df_zhys.drop(df_zhys[df_zhys[col1]=="汇客好租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="澄心优租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="CPS渠道合作"].index,inplace=True)
    
    df_zhys.drop(df_zhys[df_zhys[col1]=="趣智数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="格木木二奢名品"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="广州康基贸易有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="线下小店"].index,inplace=True)  # 计划
    df_zhys.drop(df_zhys[df_zhys[col1]=="小蚂蚁租机"].index,inplace=True)  #拒量订单


    df_zhys.drop(df_zhys[df_zhys[col2].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
    return df_zhys

df_zhys = drop_sj(df_zhys,'merchant_name','model_number')
df_zhys.shape

(27070, 63)

# 历史订单渠道名称补充

In [272]:

qudao_name_df = pd.read_excel('F:/myfile/租后数据/历史订单渠道名称补充.xlsx')
qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)

In [273]:
# dfzh1 = df_zh.merge(qudao_name_df,on = '订单号',how = 'left')
dfzh1 =  df_zhys.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号',how = 'left')
# dfzh1.columns

In [274]:
# 渠道名称缺失补充
# dfzh1["渠道名称"] = dfzh1["渠道名称_x"].fillna(dfzh1.渠道名称_y)
dfzh1["channel_name"] = np.where(dfzh1.渠道名称.notnull(),dfzh1.渠道名称,dfzh1["channel_name"])
dfzh1["channel_name"].isnull().sum()

np.int64(0)

# 渠道归属

In [275]:
def qudao_type(a,b,c):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"
    elif c == 1:
        return "芝麻租物" 
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "八派信息" in a:
        return "八派信息"
    elif "九州信息" in a:
        return "九州信息"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "支付宝直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "社群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    else :
        return a


In [276]:
# df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)

In [277]:
dfzh1["activity_source"] = dfzh1["activity_source"].fillna('未知活动') 
dfzh1["channel_name"] = dfzh1["channel_name"].fillna('未知渠道') 

dfzh1.loc[:,"归属渠道"]=dfzh1.apply(lambda x:qudao_type(x["channel_name"],x["activity_source"],x["order_method"]),axis=1)

# pd.crosstab(dfzh1["下单月份"],dfzh1["归属渠道"],margins=True)

# 判断颜色 内存

In [278]:
dfzh1["内存"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[0].str.split(" ").str[0]
# dfzh1["内存"].value_counts()[14:]

dfzh1["颜色"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[1].str.split(" ").str[0]
dfzh1["颜色"].value_counts()[1:10]

颜色
原色钛金属         4807
黑色钛金属         3395
暗紫色           1834
深空黑色          1377
蓝色钛金属         1329
黑色             903
银色             882
蓝色钛金属-现货速发     818
蓝色             605
Name: count, dtype: int64

# 判断二手

In [279]:
# dfzh1['是否二手'] = np.where(dfzh1['model_number'].str.contains('95新')|dfzh1['model_number'].str.contains('99新')|dfzh1['model_number'].str.contains('准新'),"二手","全新")
dfzh1['是否二手'] = np.where(dfzh1['product_name'].str.contains('95新')|dfzh1['product_name'].str.contains('99新')|dfzh1['product_name'].str.contains('准新'),"二手","全新")
dfzh1['是否二手'].value_counts()

是否二手
全新    24948
二手     2122
Name: count, dtype: int64

In [280]:
dfzh1[dfzh1['model_number'].str.contains('95新')]['model_number'].value_counts()

model_number
95新  iPhone 11 国行 官方质检【租物特惠】     2
 95新  iPhone 11 国行 官方质检          2
95新  iPhone 11 Pro 国行 官方质检       1
95新 iPhone14ProMax 国行正品 顺丰包邮     1
95新 iPhone 11 Pro 国行 官方质检        1
Name: count, dtype: int64

# 日期处理

In [281]:

dfzh1["下单日期"] = dfzh1['order_create_time'].dt.date
dfzh1["下单日期"] = pd.to_datetime(dfzh1["下单日期"],errors="coerce")
dfzh1["下单日期"].head()

dfzh1["月份"]=dfzh1["下单日期"].dt.month
dfzh1["年份"]=dfzh1["下单日期"].dt.year
# df_total["下单月份"].value_counts()

In [282]:
def getjidu(s):
    if s<4:
        return "1季度"
    elif 4<=s<7:
        return "2季度"
    elif 7<=s<10:
        return "3季度"
    else:
        return "4季度"
dfzh1["季度"]= dfzh1.apply(lambda x:getjidu(x["月份"]),axis=1)  
dfzh1["季度"].value_counts()

季度
3季度    9671
4季度    8450
2季度    6164
1季度    2785
Name: count, dtype: int64

# 买断数据获取

In [283]:
sql1 = ''' --   买断信息 
with out_order as ( 
SELECT too.order_id,too.`status` 
,too.real_pay_money ,too.create_time,too.update_time
,too.pay_date ,too.actual_money 
,rank() over(partition by too.order_id order by too.pay_date desc) as rn 
from db_digua_business.t_order_out too
where  too.`status` not in (1)   -- 买断状态：1 未买断 2 已买断  3 部分买断 
-- and too.pay_status= 2    -- '支付状态 1、待支付 2、已支付 3、退款中 4、已退款 6、支付取消 7、已关闭',
-- GROUP BY 1 ,2
-- ORDER BY 5 desc,1
)
SELECT t1.*
,t2.`status`
,case when t2.`status`=2 then '已买断' when  t2.`status`=3 and om.order_finish_date is not null then '已买断'  else '部分买断' end as if_outpay 
,om.order_finish_date
,case when t2.`status`=2 and om.order_finish_date is null then t1.pay_date else om.order_finish_date end as finish_date_new 
from (
SELECT too.order_id  -- ,too.`status` 
-- ,count(DISTINCT too.`status`) nn 
,sum(too.real_pay_money) as outpay_money     -- ,max(too.update_time) as outpay_time
,max(too.pay_date) as pay_date 
from out_order  too
GROUP BY 1 -- ,2
ORDER BY 2 desc ,1) t1 
left join out_order t2 on t1.order_id = t2.order_id and t2.rn=1 
left join db_digua_business.t_order om on t1.order_id = om.id 
;
'''

df_out = query(sql1)
df_out.shape

(2514, 7)

In [284]:
df_out.columns

Index(['order_id', 'outpay_money', 'pay_date', 'status', 'if_outpay',
       'order_finish_date', 'finish_date_new'],
      dtype='object')

# 租后数据 关联 买断数据

In [285]:
dfzh2 = dfzh1.merge(df_out[['order_id', 'outpay_money', 'pay_date', 'status', 'if_outpay']], on = 'order_id', how = 'left')
dfzh2.columns

Index(['id', 'order_id', 'order_number', 'order_time', 'search_time',
       'end_date', 'merchant_id', 'merchant_name', 'channel_id',
       'channel_name', 'model_number', 'specification', 'purchase_amount',
       'platform_package', 'all_deposit', 'all_rental', 'liquidated_damages',
       'buyout_balance_payment', 'advance_periods', 'advance_sum',
       'order_create_time', 'rembursement_status', 'paid_periods',
       'begin_overdue_time', 'overdue_periods', 'max_overdue_days',
       'now_overdue_days', 'total_receivable', 'contract_price',
       'rent_received', 'unearned_rent_revenue', 'paid_buyout_balance_payment',
       'total_received', 'total_unreceived', 'unrecovered_cost',
       'overdue_type', 'gross_profit_receivable', 'overdue_loss',
       'actual_gross_profit', 'overdue_total_unreceived', 'relet_periods',
       'relet_days', 'relet_rent_paid', 'activity_id', 'activity_source',
       'order_label', 'create_time', 'update_time', 'order_finish_date',
       'comm

In [286]:
# dfzh2[['order_finish_date','订单完成时间','pay_date','status']].info()

In [287]:
dfzh2['finish_date_new'] = dfzh2.apply(lambda x:pd.to_datetime(x.pay_date, errors='coerce') if pd.isnull(x.order_finish_date) and x.status==2
                                       else pd.to_datetime(x.订单完成时间, errors='coerce') if pd.isnull(x.order_finish_date)  and pd.notnull(x.订单完成时间)
                                       else pd.to_datetime(x.order_finish_date, errors='coerce'),axis = 1 ) 
# dfzh2['finish_date_new'] = np.where((pd.isnull(dfzh2.order_finish_date)) & (dfzh2.status==2),pd.to_datetime(dfzh2.pay_date, errors='coerce'),pd.to_datetime(dfzh2.order_finish_date, errors='coerce'))
dfzh2['finish_date_new'].notnull().sum()

np.int64(3387)

In [288]:

dfzh2['if_outpay'] = dfzh2.apply(lambda x: '已买断' if x['status'] ==2 
                                 else  '已买断'  if  x['status'] ==3 and pd.notnull(x.finish_date_new) 
                                 else '部分买断' if x['status'] ==3 and pd.isnull(x.finish_date_new) 
                                 else '未知',axis =1)
dfzh2['if_outpay'].value_counts()


if_outpay
未知      24647
已买断      2374
部分买断       56
Name: count, dtype: int64

# 分期 取消数据 获取

In [289]:
sql1 = '''-- 订单分期表  取消的分期
SELECT tos.order_id,count(*) as nn
,count(tos.reality_refund_date) as  reality_refund_date
,sum(tos.real_pay_money) as real_pay_money
,sum(tprm.purchase_amount)/count(tos.order_id) as purchase_amount
,avg(tprm.advance_periods) as  advance_periods
,avg(tprm.advance_sum) as advance_sum
from db_digua_business.t_order_stages tos 
inner join db_digua_business.t_postlease_receivables_monitoring   tprm 
on tos.order_id = tprm.order_id 
where tos.`status`=4 
GROUP BY 1 order by 2 desc 
;
'''

df_cancel = query(sql1)
df_cancel.shape

(1500, 7)

In [290]:
# df_cancel.columns

# 租后数据 关联 取消数据

In [291]:
dfzh3 = dfzh2.merge(df_cancel[['order_id', 'nn', 'reality_refund_date', 'real_pay_money']],on = 'order_id',how = 'left')
dfzh3.columns

Index(['id', 'order_id', 'order_number', 'order_time', 'search_time',
       'end_date', 'merchant_id', 'merchant_name', 'channel_id',
       'channel_name', 'model_number', 'specification', 'purchase_amount',
       'platform_package', 'all_deposit', 'all_rental', 'liquidated_damages',
       'buyout_balance_payment', 'advance_periods', 'advance_sum',
       'order_create_time', 'rembursement_status', 'paid_periods',
       'begin_overdue_time', 'overdue_periods', 'max_overdue_days',
       'now_overdue_days', 'total_receivable', 'contract_price',
       'rent_received', 'unearned_rent_revenue', 'paid_buyout_balance_payment',
       'total_received', 'total_unreceived', 'unrecovered_cost',
       'overdue_type', 'gross_profit_receivable', 'overdue_loss',
       'actual_gross_profit', 'overdue_total_unreceived', 'relet_periods',
       'relet_days', 'relet_rent_paid', 'activity_id', 'activity_source',
       'order_label', 'create_time', 'update_time', 'order_finish_date',
       'comm

In [292]:
# 不同还款状态下  完成时间的情况
# dfzh3.groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

In [293]:
# 不同订单状态下  完成时间的情况
# dfzh3.groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

In [294]:
# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})


In [295]:
# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})


# 按风控策略调整时间节点划分时间段

In [296]:
#按风控策略调整时间节点划分时间段
dfzh1.loc[:"2023-01-09","时间段"]=" -2023.1.9前"
dfzh1.loc["2023-01-10":"2023-03-03","时间段"]="2023.1.10-2023.3.3"
dfzh1.loc["2023-03-04":"2023-03-24","时间段"]="2023.3.4-2023.3.24"
dfzh1.loc["2023-03-25":"2023-04-28","时间段"]="2023.3.25-2023.4.28"
dfzh1.loc["2023-04-29":"2023-05-17","时间段"]="2023.4.29-2023.5.17"
dfzh1.loc["2023-05-18":"2023-05-31","时间段"]="2023.5.18-2023.5.31"
dfzh1.loc["2023-06-01":"2023-06-30","时间段"]="2023.6.1-2023.6.30"
dfzh1.loc["2023-07-01":"2023-08-11","时间段"]="2023.7.1-2023.8.11"
dfzh1.loc["2023-08-12":"2023-09-15","时间段"]="2023.8.12-2023.9.15"
dfzh1.loc["2023-09-16":"2023-09-22","时间段"]="2023.9.16-2023.9.22"
dfzh1.loc["2023-09-23":"2023-11-16","时间段"]="2023.9.23-2023.11.16"
dfzh1.loc["2023-11-17":"2023-11-28","时间段"]="2023.11.17-2023.11.28"
dfzh1.loc["2023-11-29":"2023-12-05","时间段"]="2023.11.29-2023.12.5"
dfzh1.loc["2023-12-06":"2024-01-05","时间段"]="2023.12.06-2024.1.5"
dfzh1.loc["2024-01-06":"2024-01-23","时间段"]="2024.1.6-2024.1.23"
dfzh1.loc["2024-01-24":"2024-2-21","时间段"]="2024.1.24-2024.2.21"
dfzh1.loc["2024-02-22":"2024-02-29","时间段"]="2024.2.22-2024.2.29"
dfzh1.loc["2024-03-01":"2024-03-12","时间段"]="2024.3.1-2024.3.12"
dfzh1.loc["2024-03-13":"2024-04-03","时间段"]="2024.3.13-2024.4.3"
dfzh1.loc["2024-04-04":"2024-04-10","时间段"]="2024.4.4-2024.4.10"
dfzh1.loc["2024-04-11":"2024-04-18","时间段"]="2024.4.11-2024.4.18"
dfzh1.loc["2024-04-19":"2024-04-24","时间段"]="2024.4.19-2024.4.24"
dfzh1.loc["2024-04-25":"2024-04-27","时间段"]="2024.4.25-2024.4.27"
dfzh1.loc["2024-04-28":,"时间段"]="2024.4.28- "

# 账期数据提取   含续租

## 获取续租分期数据

In [297]:
sql_xz = ''' 
-- 状态,1:未支付  2:扣款失败 3:已支付 4：已取消 5、申请扣款、6已退款',
-- '是否续租账期,0非续租账期，1预授权账期，2自定义续租账期',
SELECT ymos.*
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
;
'''

df_xzfq = query(sql_xz)
df_xzfq.shape

(357789, 23)

## 重命名

In [298]:
df_xzfq = df_xzfq.rename(columns={'money':'当前应付租金','part_payment':'实付金额','refund_date':'应付日期'
,'reality_refund_date':'实付日期','sort':'当前期数'})

In [299]:
# df_ddfq = df_ddfq.rename(columns={'排序还款期数':'当前期数'})
# df_ddfq.columns

In [300]:
# df_xzfq.columns

## 实还金额核查

In [301]:
# total_check = df_ddfq.merge(df_xzfq,on=['订单ID','当前期数'],how='inner')
# total_check.info()

In [302]:
# total_check1= total_check[(total_check.实付金额_y < total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check1= total_check[(total_check.实付金额 < total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check1

In [303]:
# total_check2= total_check[(total_check.实付金额_y > total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())

# total_check2= total_check[(total_check.实付金额 > total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check2

In [304]:
# total_check2.to_excel('E:/myfile/业务相关数据/total_check_0709-1.xlsx')

# 数据合并

In [305]:
df_xz_concat = df_xzfq.merge(dfzh3,how='right',on ='order_id')
# df_xz_concat = df_fq.merge(dfzh1,how='right',right_on ='订单id',left_on='订单ID')

df_xz_concat['当前期数'].notnull().sum()

np.int64(357686)

In [306]:
df_xz_concat['order_id'].nunique()

27070

In [307]:
dfzh3['order_id'].nunique()

27070

In [308]:
df_xz_concat[["order_id","当前期数"]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357690 entries, 0 to 357689
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   order_id  357690 non-null  int64  
 1   当前期数      357686 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 5.5 MB


# 去重

In [309]:
check = df_xz_concat[["order_id","当前期数"]]  # ,"逾期天数"
all_duplicates = check[check.duplicated(keep=False)]  
# all_duplicates.to_excel('E:/myfile/业务相关数据/all_duplicates.xlsx')
print(all_duplicates.sort_values(by=['order_id','当前期数']))

       order_id  当前期数
8063     119321   1.0
8075     119321   1.0
8064     119321   2.0
8076     119321   2.0
8065     119321   3.0
...         ...   ...
65318    552441  10.0
65307    552441  11.0
65319    552441  11.0
65308    552441  12.0
65320    552441  12.0

[168 rows x 2 columns]


In [310]:
#删除重复订单
df_xz_concat.drop_duplicates(subset=["order_id","当前期数"],inplace=True)

In [311]:
# df_xz_concat.to_excel('E:/myfile/业务相关数据/df_xz_concat.xlsx')

## 异常查询

In [312]:
df_xz_concat.columns

Index(['id_x', 'order_id', '当前应付租金', '实付金额', 'interest_money', '应付日期', '实付日期',
       'status_x', '当前期数', 'order_pay_id',
       ...
       '年份', '季度', 'outpay_money', 'pay_date', 'status_y', 'if_outpay',
       'finish_date_new', 'nn', 'reality_refund_date', 'real_pay_money'],
      dtype='object', length=103)

In [313]:
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.status_x==1)][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

实付金额>0  实付日期为空  finish_date_new非空  :实付日期=订单完成时间

In [314]:
df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

,order_number,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new
2776,A202211111913512,650.70,79.00,2023-09-11,NaT,1.0,11.0,已逾期,NaT,5,NaT
4880,A202301071415408,915.00,200.00,2024-05-11,NaT,1.0,17.0,已逾期,NaT,4,NaT
10587,A2023022108454753,641.86,320.84,2023-12-24,NaT,1.0,11.0,还款中,NaT,5,NaT
14918,A2023032515133228,1499.22,865.00,2024-07-27,NaT,1.0,17.0,已逾期,NaT,4,NaT
15571,A2023033003351727,1084.73,700.00,2024-07-01,NaT,1.0,16.0,已逾期,NaT,4,NaT
25146,A2023050408232337,670.85,412.63,2024-12-06,NaT,1.0,20.0,续租中,NaT,4,NaT
40685,A20230627115744111,479.73,201.35,2024-03-30,NaT,1.0,10.0,还款中,NaT,5,NaT
42030,A2023070115535442,446.07,53.93,2024-05-04,NaT,1.0,11.0,还款中,NaT,5,NaT
61036,A2023081019051714,1074.81,700.00,2024-07-13,NaT,1.0,12.0,已买断,NaT,8,NaT
114352,A20231112005735116,732.98,368.38,2025-01-14,NaT,1.0,15.0,续租中,NaT,4,NaT


part_payment==0  reality_refund_date.notnull()  还款状态== '续租中' part_payment=money   还款状态== '已逾期' reality_refund_date=pd.na

In [315]:
dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]

dd_test

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn
102,A202207181530165,9777,306.60,0.0,2023-01-18,2023-03-18 19:54:59,2.0,7.0,已逾期,NaT,5,NaT,NaN
103,A202207181530165,9777,306.60,0.0,2023-02-18,2023-03-18 19:55:00,2.0,8.0,已逾期,NaT,5,NaT,NaN
104,A202207181530165,9777,306.60,0.0,2023-03-18,2023-03-18 19:55:01,2.0,9.0,已逾期,NaT,5,NaT,NaN
137,A202207200249265,10056,736.56,0.0,2022-12-20,2023-03-18 19:55:02,2.0,6.0,已逾期,NaT,4,NaT,NaN
138,A202207200249265,10056,736.56,0.0,2023-01-20,2023-03-18 19:55:03,2.0,7.0,已逾期,NaT,4,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
357354,A20241204164922353,1812057,0.00,0.0,2024-12-07,2024-12-04 17:09:53,3.0,1.0,未到首期还款日,NaT,3,NaT,NaN
357426,A20241204172409143,1812257,0.00,0.0,2024-12-07,2024-12-04 17:27:20,3.0,1.0,未到首期还款日,NaT,3,NaT,NaN
357486,A20241204174056237,1812351,0.00,0.0,2024-12-07,2024-12-04 17:50:01,3.0,1.0,未到首期还款日,NaT,3,NaT,NaN
357546,A202412041801129,1812460,0.00,0.0,2024-12-07,2024-12-04 18:22:27,3.0,1.0,未到首期还款日,NaT,3,NaT,NaN


In [316]:
dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
dd_test

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn
102,A202207181530165,9777,306.60,0.0,2023-01-18,2023-03-18 19:54:59,2.0,7.0,已逾期,NaT,5,NaT,NaN
103,A202207181530165,9777,306.60,0.0,2023-02-18,2023-03-18 19:55:00,2.0,8.0,已逾期,NaT,5,NaT,NaN
104,A202207181530165,9777,306.60,0.0,2023-03-18,2023-03-18 19:55:01,2.0,9.0,已逾期,NaT,5,NaT,NaN
137,A202207200249265,10056,736.56,0.0,2022-12-20,2023-03-18 19:55:02,2.0,6.0,已逾期,NaT,4,NaT,NaN
138,A202207200249265,10056,736.56,0.0,2023-01-20,2023-03-18 19:55:03,2.0,7.0,已逾期,NaT,4,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
357354,A20241204164922353,1812057,0.00,0.0,2024-12-07,2024-12-04 17:09:53,3.0,1.0,未到首期还款日,NaT,3,NaT,NaN
357426,A20241204172409143,1812257,0.00,0.0,2024-12-07,2024-12-04 17:27:20,3.0,1.0,未到首期还款日,NaT,3,NaT,NaN
357486,A20241204174056237,1812351,0.00,0.0,2024-12-07,2024-12-04 17:50:01,3.0,1.0,未到首期还款日,NaT,3,NaT,NaN
357546,A202412041801129,1812460,0.00,0.0,2024-12-07,2024-12-04 18:22:27,3.0,1.0,未到首期还款日,NaT,3,NaT,NaN


实付为0，实付日期非空，完成时间为空，无期数取消：实付日期清空

In [317]:
dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
dd_test.订单状态值.value_counts()

订单状态值
4    236
5    174
3     65
2      2
Name: count, dtype: int64

实付日期为空，完成时间为空，取消期数>=1 ：状态=已取消

In [318]:
dd_test = df_xz_concat[ (df_xz_concat.实付日期.isnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn>=1)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
dd_test.订单状态值.value_counts()

订单状态值
10    3133
8      204
4       11
5        8
Name: count, dtype: int64

部分还款，实付日期非空，完成时间为空 用户：  实付日期清空  

In [319]:
dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]

dd_test1

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn
1980,A202210092154098,27461,591.60,408.40,2023-01-09 00:00:00,2023-03-18 20:04:40,2.0,4.0,已逾期,NaT,5,NaT,NaN
49257,A2023071909113511,477555,583.26,200.00,2024-02-22 00:00:00,2024-02-28 20:07:18,1.0,8.0,已逾期,NaT,5,NaT,NaN
126713,A2023112606505333,814549,541.32,22.76,2024-08-29 00:00:00,2024-08-12 17:47:02,1.0,10.0,已逾期,NaT,4,NaT,NaN
287428,A20240915113026156,1475789,544.92,534.02,2024-11-18 00:00:00,2024-11-14 00:37:19,3.0,3.0,还款中,NaT,4,NaT,NaN
290704,A20240914145305347,1471829,499.85,474.85,2024-11-23 22:25:25,2024-11-23 21:08:24,3.0,3.0,还款中,NaT,4,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
356081,A20241203174852293,1807395,512.38,488.81,2025-11-06 00:00:00,2024-12-03 18:54:39,3.0,12.0,未到首期还款日,NaT,3,NaT,NaN
356213,A20241203205225112,1807981,461.63,440.49,2025-11-06 00:00:00,2024-12-03 20:58:25,3.0,12.0,未到首期还款日,NaT,3,NaT,NaN
356309,A2024120318093452,1807499,474.90,467.04,2025-11-06 00:00:00,2024-12-04 13:52:24,3.0,12.0,未到首期还款日,NaT,3,NaT,NaN
356537,A2024120410142674,1809232,421.89,421.88,2025-11-07 00:00:00,2024-12-04 10:21:15,3.0,12.0,未到首期还款日,NaT,3,NaT,NaN


 部分还款，实付日期为空，完成时间非空 用户：  实付日期=完成时间

In [320]:
dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]

dd_test1.订单状态值.value_counts()

订单状态值
8    162
Name: count, dtype: int64

已买断 但无实付日期及金额：实付日回写

In [321]:
dd_test1 = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

# dd_test1.订单状态值.value_counts()
dd_test1

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money
22,A202207050851373,8924,354.00,0.0,2024-05-01,NaT,1.0,23.0,已买断,2024-04-17 01:37:33,8,2024-04-17 01:37:33,NaN,329.72
23,A202207050851373,8924,47.20,0.0,2024-05-31,NaT,1.0,24.0,已买断,2024-04-17 01:37:33,8,2024-04-17 01:37:33,NaN,329.72
419,A202208032116483,12219,483.00,0.0,2023-12-31,NaT,1.0,18.0,已买断,2023-11-06 10:12:57,8,2023-11-06 10:12:57,NaN,2709.20
420,A202208032116483,12219,483.00,0.0,2024-01-30,NaT,1.0,19.0,已买断,2023-11-06 10:12:57,8,2023-11-06 10:12:57,NaN,2709.20
421,A202208032116483,12219,483.00,0.0,2024-02-29,NaT,1.0,20.0,已买断,2023-11-06 10:12:57,8,2023-11-06 10:12:57,NaN,2709.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174463,A2024041316125534,995895,1061.16,0.0,2024-11-16,NaT,1.0,8.0,已买断,2024-05-17 15:02:17,8,2024-05-17 15:02:17,10.0,10014.38
174464,A2024041316125534,995895,1061.16,0.0,2024-12-16,NaT,1.0,9.0,已买断,2024-05-17 15:02:17,8,2024-05-17 15:02:17,10.0,10014.38
174465,A2024041316125534,995895,1061.16,0.0,2025-01-16,NaT,1.0,10.0,已买断,2024-05-17 15:02:17,8,2024-05-17 15:02:17,10.0,10014.38
174466,A2024041316125534,995895,1061.16,0.0,2025-02-16,NaT,1.0,11.0,已买断,2024-05-17 15:02:17,8,2024-05-17 15:02:17,10.0,10014.38


##  实还日期 修正

In [322]:
df_xz_concat['实付日期'] = pd.to_datetime(df_xz_concat['实付日期']).dt.date
df_xz_concat['finish_date_new'] = pd.to_datetime(df_xz_concat['finish_date_new']).dt.date

In [323]:
# 部分还款 用户 实付日期非空 且非买断 非完成 ： 实付日期清空  
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull()),pd.NaT,df_xz_concat.实付日期)
df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']]                            

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
1980,A202210092154098,27461,591.60,408.40,2023-01-09 00:00:00,2023-03-18,2.0,4.0,已逾期,NaT,5,NaT,NaN,NaT
49257,A2023071909113511,477555,583.26,200.00,2024-02-22 00:00:00,2024-02-28,1.0,8.0,已逾期,NaT,5,NaT,NaN,NaT
126713,A2023112606505333,814549,541.32,22.76,2024-08-29 00:00:00,2024-08-12,1.0,10.0,已逾期,NaT,4,NaT,NaN,NaT
287428,A20240915113026156,1475789,544.92,534.02,2024-11-18 00:00:00,2024-11-14,3.0,3.0,还款中,NaT,4,NaT,NaN,NaT
290704,A20240914145305347,1471829,499.85,474.85,2024-11-23 22:25:25,2024-11-23,3.0,3.0,还款中,NaT,4,NaT,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356081,A20241203174852293,1807395,512.38,488.81,2025-11-06 00:00:00,2024-12-03,3.0,12.0,未到首期还款日,NaT,3,NaT,NaN,NaT
356213,A20241203205225112,1807981,461.63,440.49,2025-11-06 00:00:00,2024-12-03,3.0,12.0,未到首期还款日,NaT,3,NaT,NaN,NaT
356309,A2024120318093452,1807499,474.90,467.04,2025-11-06 00:00:00,2024-12-04,3.0,12.0,未到首期还款日,NaT,3,NaT,NaN,NaT
356537,A2024120410142674,1809232,421.89,421.88,2025-11-07 00:00:00,2024-12-04,3.0,12.0,未到首期还款日,NaT,3,NaT,NaN,NaT


In [324]:
# 已完成 还款 用户  实付日期 补充 订单完成时间       实付金额>0  实付日期为空  finish_date_new非空  :实付日期=订单完成时间finish_date_new
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)
df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']]    

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
1980,A202210092154098,27461,591.60,408.40,2023-01-09,2023-03-18,2.0,4.0,已逾期,NaT,5,NaT,NaN,NaT
2776,A202211111913512,35537,650.70,79.00,2023-09-11,NaT,1.0,11.0,已逾期,NaT,5,NaT,NaN,NaT
4880,A202301071415408,87888,915.00,200.00,2024-05-11,NaT,1.0,17.0,已逾期,NaT,4,NaT,NaN,NaT
10587,A2023022108454753,144517,641.86,320.84,2023-12-24,NaT,1.0,11.0,还款中,NaT,5,NaT,9.0,NaT
14918,A2023032515133228,193214,1499.22,865.00,2024-07-27,NaT,1.0,17.0,已逾期,NaT,4,NaT,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356081,A20241203174852293,1807395,512.38,488.81,2025-11-06,2024-12-03,3.0,12.0,未到首期还款日,NaT,3,NaT,NaN,NaT
356213,A20241203205225112,1807981,461.63,440.49,2025-11-06,2024-12-03,3.0,12.0,未到首期还款日,NaT,3,NaT,NaN,NaT
356309,A2024120318093452,1807499,474.90,467.04,2025-11-06,2024-12-04,3.0,12.0,未到首期还款日,NaT,3,NaT,NaN,NaT
356537,A2024120410142674,1809232,421.89,421.88,2025-11-07,2024-12-04,3.0,12.0,未到首期还款日,NaT,3,NaT,NaN,NaT


In [325]:
# 租完即送用户 A2023081019051714 补还款时间 
df_xz_concat['实付日期1'] = np.where((df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12),'2024-07-22',df_xz_concat['实付日期1'])
df_xz_concat[(df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12)][['实付日期1']]

,实付日期1
61036,2024-07-22


In [326]:
# 未还款 且 未完成 未取消用户 ： 实付日期清空
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull()),pd.NaT,df_xz_concat.实付日期1)
df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']] 


,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
102,A202207181530165,9777,306.60,0.0,2023-01-18,2023-03-18,2.0,7.0,已逾期,NaT,5,NaT,NaN,NaT
103,A202207181530165,9777,306.60,0.0,2023-02-18,2023-03-18,2.0,8.0,已逾期,NaT,5,NaT,NaN,NaT
104,A202207181530165,9777,306.60,0.0,2023-03-18,2023-03-18,2.0,9.0,已逾期,NaT,5,NaT,NaN,NaT
137,A202207200249265,10056,736.56,0.0,2022-12-20,2023-03-18,2.0,6.0,已逾期,NaT,4,NaT,NaN,NaT
138,A202207200249265,10056,736.56,0.0,2023-01-20,2023-03-18,2.0,7.0,已逾期,NaT,4,NaT,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357354,A20241204164922353,1812057,0.00,0.0,2024-12-07,2024-12-04,3.0,1.0,未到首期还款日,NaT,3,NaT,NaN,NaT
357426,A20241204172409143,1812257,0.00,0.0,2024-12-07,2024-12-04,3.0,1.0,未到首期还款日,NaT,3,NaT,NaN,NaT
357486,A20241204174056237,1812351,0.00,0.0,2024-12-07,2024-12-04,3.0,1.0,未到首期还款日,NaT,3,NaT,NaN,NaT
357546,A202412041801129,1812460,0.00,0.0,2024-12-07,2024-12-04,3.0,1.0,未到首期还款日,NaT,3,NaT,NaN,NaT


In [327]:
# 实付为0，实付日期为空，但 完成时间非空，买断金额>0： 实付日期回写

df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)
df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.notnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']] 

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
22,A202207050851373,8924,354.00,0.0,2024-05-01,NaT,1.0,23.0,已买断,2024-04-17 01:37:33,8,2024-04-17,NaN,2024-04-17
23,A202207050851373,8924,47.20,0.0,2024-05-31,NaT,1.0,24.0,已买断,2024-04-17 01:37:33,8,2024-04-17,NaN,2024-04-17
419,A202208032116483,12219,483.00,0.0,2023-12-31,NaT,1.0,18.0,已买断,2023-11-06 10:12:57,8,2023-11-06,NaN,2023-11-06
420,A202208032116483,12219,483.00,0.0,2024-01-30,NaT,1.0,19.0,已买断,2023-11-06 10:12:57,8,2023-11-06,NaN,2023-11-06
421,A202208032116483,12219,483.00,0.0,2024-02-29,NaT,1.0,20.0,已买断,2023-11-06 10:12:57,8,2023-11-06,NaN,2023-11-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174463,A2024041316125534,995895,1061.16,0.0,2024-11-16,NaT,1.0,8.0,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17
174464,A2024041316125534,995895,1061.16,0.0,2024-12-16,NaT,1.0,9.0,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17
174465,A2024041316125534,995895,1061.16,0.0,2025-01-16,NaT,1.0,10.0,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17
174466,A2024041316125534,995895,1061.16,0.0,2025-02-16,NaT,1.0,11.0,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17


In [328]:
# 部分还款 
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [329]:
# 未完成 但含有分期取消的 订单
# check = df_xz_concat[(df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull()) & df_xz_concat.nn>=1][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# check.订单状态值.value_counts()

In [330]:
# 实付0，实付日期为空，但 完成时间非空：为取消订单，实付日期=完成时间
df_xz_concat['实付日期new'] = np.where( (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)
df_xz_concat[ (df_xz_concat.实付日期new.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期1,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [331]:
# 属于部分还款 :无实付日期
# dd_test2 = df_xz_concat[(df_xz_concat.实付金额 >0) & (df_xz_concat.实付日期new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# dd_test2

In [332]:
# df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [333]:
# A202207050851373 应付日期修复
df_xz_concat['应付日期'] = np.where((df_xz_concat.应付日期.isnull()) & (df_xz_concat.order_number=='A202207050851373'),pd.to_datetime('2023-07-05') ,df_xz_concat.应付日期 )
df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期new,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money
326666,A20241104144812228,1681669,NaN,NaN,None,NaT,NaN,NaN,未到首期还款日,NaT,13,NaT,NaN,NaN
349335,A20241127172811165,1780189,NaN,NaN,None,NaT,NaN,NaN,未到首期还款日,NaT,10,NaT,12.0,NaN
350404,A20241126174415266,1775399,NaN,NaN,None,NaT,NaN,NaN,未到首期还款日,NaT,4,NaT,NaN,NaN
353417,A20241201162335145,1796917,NaN,NaN,None,NaT,NaN,NaN,未到首期还款日,NaT,3,NaT,NaN,NaN


In [334]:
# 实付金额>0  实付日期.isnull()  还款状态==已买断  :  逾期天数=0  实付日期=订单完成时间
# 实付金额>0  实付日期.isnull()  还款状态==已逾期  :实付金额 =0  
# 实付金额==0  实付日期.notnull()  还款状态== '续租中' 实付金额=当前应付租金   还款状态== '已逾期' 实付日期=pd.na
# 0<实付金额<当前应付租金  实付日期.notnull()  还款状态== '已逾期'  :  实付金额=0  & 实付日期=pd.na

# df_xz_concat['实付金额new'] = df_xz_concat.apply(lambda x:
#     0 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已逾期'
#     else 0 if x.实付金额>0 and x.实付金额<x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.当前应付租金 if x.实付金额==0 and pd.notna(x.实付日期) and x.还款状态=='续租中'
#     else x.实付金额
#     ,axis = 1)



In [335]:
# df_xz_concat['实付日期new'] = df_xz_concat.apply(lambda x:
#     x.订单完成时间 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已买断'
#     else np.nan if x.实付金额>=0 and x.实付金额 < x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.实付日期
#     ,axis = 1)

## 获取逾期截止时间

In [336]:
df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['now_date']

0        2024-12-05
1        2024-12-05
2        2024-12-05
3        2024-12-05
4        2024-12-05
            ...    
357685   2024-12-05
357686   2024-12-05
357687   2024-12-05
357688   2024-12-05
357689   2024-12-05
Name: now_date, Length: 357606, dtype: datetime64[s]

## 重新定义状态

In [337]:
# df_xz_concat.columns

In [338]:
df_xz_concat[(df_xz_concat.实付日期new.isnull()) & (~df_xz_concat.订单状态值.isin([8,10]))][['rembursement_status']].value_counts()

rembursement_status
还款中                    70949
已逾期                    34101
未到首期还款日                25056
续租中                    17780
检测中                       34
Name: count, dtype: int64

In [339]:
df_xz_concat[(df_xz_concat.实付日期new.isnull())& (pd.isna(df_xz_concat.finish_date_new))][['订单状态']].value_counts()

订单状态
租赁中     138428
待归还       6211
已退款       3133
待收货       2887
待发货        359
已完成        214
检测中         34
订单取消         1
Name: count, dtype: int64

In [340]:
conditions = [  
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.isna(df_xz_concat['finish_date_new'])) & (df_xz_concat['订单状态值'].isin([8, 10]))),  
    ((pd.to_datetime(df_xz_concat['now_date']) <= pd.to_datetime(df_xz_concat['应付日期'])) ),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) <= pd.to_datetime(df_xz_concat['应付日期']))),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) > pd.to_datetime(df_xz_concat['应付日期']))) ,
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['now_date']) > pd.to_datetime(df_xz_concat['应付日期'])) )  

]  
  
choices = ['已取消', '未到还款日', '正常还款', '已逾期支付','已逾期']  
  
# 使用numpy.where来应用条件  
df_xz_concat['状态'] = np.select(conditions, choices, default='其他')

df_xz_concat['状态'].value_counts()

状态
正常还款     157976
未到还款日    149670
已逾期支付     24847
已逾期       21763
已取消        3347
其他            3
Name: count, dtype: int64

In [341]:
df_xz_concat.head()

,id_x,order_id,当前应付租金,实付金额,interest_money,应付日期,实付日期,status_x,当前期数,order_pay_id,...,status_y,if_outpay,finish_date_new,nn,reality_refund_date,real_pay_money,实付日期1,实付日期new,now_date,状态
0,6506.0,8924,357.95,357.95,0.0,1656979200000000000,2022-07-05,3.0,1.0,9576,...,2.0,已买断,2024-04-17,NaN,NaN,NaN,2022-07-05,2022-07-05,2024-12-05,正常还款
1,6507.0,8924,357.95,357.95,0.0,1659657600000000000,2022-08-05,3.0,2.0,9576,...,2.0,已买断,2024-04-17,NaN,NaN,NaN,2022-08-05,2022-08-05,2024-12-05,正常还款
2,6508.0,8924,357.95,357.95,0.0,1662336000000000000,2022-09-05,3.0,3.0,9576,...,2.0,已买断,2024-04-17,NaN,NaN,NaN,2022-09-05,2022-09-05,2024-12-05,正常还款
3,6509.0,8924,357.95,357.95,0.0,1664928000000000000,2022-10-09,3.0,4.0,9576,...,2.0,已买断,2024-04-17,NaN,NaN,NaN,2022-10-09,2022-10-09,2024-12-05,已逾期支付
4,6510.0,8924,357.95,357.95,0.0,1667606400000000000,2022-11-08,3.0,5.0,9576,...,2.0,已买断,2024-04-17,NaN,NaN,NaN,2022-11-08,2022-11-08,2024-12-05,已逾期支付


In [342]:
df_xz_concat.shape

(357606, 107)

In [343]:
df_xz_concat.实付日期new.value_counts()

实付日期new
2024-10-15    889
2024-11-15    883
2024-11-25    831
2024-12-04    782
2024-11-26    776
             ... 
2022-09-03      1
2022-07-21      1
2022-07-05      1
2024-07-22      1
2025-04-04      1
Name: count, Length: 875, dtype: int64

In [344]:
df_xz_concat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 357606 entries, 0 to 357689
Columns: 107 entries, id_x to 状态
dtypes: datetime64[ns](12), datetime64[s](1), float64(43), int32(2), int64(15), object(34)
memory usage: 291.9+ MB


In [345]:
# df_xz_concat['实付日期new'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')
# df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')

In [346]:
# # E:/myfile/业务相关数据/风控相关数据输出_{Today}.xlsx
# df_xz_concat.to_excel('E:/myfile/业务相关数据/报错表0902.xlsx')

## 重新定义逾期天数

In [347]:
df_xz_concat['实付日期new'].isnull().sum()

np.int64(151267)

In [348]:
empty_rows = df_xz_concat[df_xz_concat['应付日期'].isna()]
empty_rows.order_number  

326666    A20241104144812228
349335    A20241127172811165
350404    A20241126174415266
353417    A20241201162335145
Name: order_number, dtype: object

In [349]:
empty_rows.order_number 

326666    A20241104144812228
349335    A20241127172811165
350404    A20241126174415266
353417    A20241201162335145
Name: order_number, dtype: object

In [350]:
df_xz_concat.columns

Index(['id_x', 'order_id', '当前应付租金', '实付金额', 'interest_money', '应付日期', '实付日期',
       'status_x', '当前期数', 'order_pay_id',
       ...
       'status_y', 'if_outpay', 'finish_date_new', 'nn', 'reality_refund_date',
       'real_pay_money', '实付日期1', '实付日期new', 'now_date', '状态'],
      dtype='object', length=107)

In [351]:
df_xz_concat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 357606 entries, 0 to 357689
Columns: 107 entries, id_x to 状态
dtypes: datetime64[ns](12), datetime64[s](1), float64(43), int32(2), int64(15), object(34)
memory usage: 291.9+ MB


In [352]:
df_xz_concat.drop

<bound method DataFrame.drop of              id_x  order_id  当前应付租金    实付金额  interest_money  \
0          6506.0      8924  357.95  357.95             0.0   
1          6507.0      8924  357.95  357.95             0.0   
2          6508.0      8924  357.95  357.95             0.0   
3          6509.0      8924  357.95  357.95             0.0   
4          6510.0      8924  357.95  357.95             0.0   
...           ...       ...     ...     ...             ...   
357685  1169796.0   1813045  695.21    0.00             0.0   
357686  1169797.0   1813045  695.21    0.00             0.0   
357687  1169798.0   1813045  695.21    0.00             0.0   
357688  1169799.0   1813045  695.21    0.00             0.0   
357689  1169800.0   1813045  696.89  696.89             0.0   

                       应付日期        实付日期  status_x  当前期数  \
0       1656979200000000000  2022-07-05       3.0   1.0   
1       1659657600000000000  2022-08-05       3.0   2.0   
2       1662336000000000000  2022-

In [353]:
# 异常值删除，应付时间为空
df_xz_concat=df_xz_concat[~df_xz_concat['应付日期'].isnull()]
df_xz_concat.shape

(357602, 107)

In [354]:

#实付日期为空且完成时间为空  订单状态为已退款或已完成  :  逾期天数=0 

df_xz_concat['逾期天数'] = df_xz_concat.apply(lambda x:
    0 if x['状态'] in ['未到还款日','正常还款','已取消']  
    else (pd.to_datetime(x['实付日期new']) - pd.to_datetime(x['应付日期'])).days  if pd.to_datetime(x['实付日期new']) >= pd.to_datetime(x['应付日期'])
    else  (pd.to_datetime(x['now_date']) - pd.to_datetime(x['应付日期'])).days   if   pd.to_datetime(x['now_date']) > pd.to_datetime(x['应付日期']) and pd.isna(x['实付日期new'])
    else 0 
    ,  axis=1)

df_xz_concat['逾期天数'].value_counts()

逾期天数
0      311005
1        5396
2        2970
3        2405
4        2172
        ...  
817         1
787         1
600         1
756         1
726         1
Name: count, Length: 740, dtype: int64

In [355]:

# liudan
# # 定义一个函数来计算逾期天数
# def calculate_overdue_days(row):
#     if row['状态'] in ['未到还款日', '正常还款', '已取消']:
#         return 0
    
#     # 转换日期为 datetime 类型，并处理 None 和 NaN
#     实付日期new = pd.to_datetime(row['实付日期new'], errors='coerce')
#     应付日期 = pd.to_datetime(row['应付日期'], errors='coerce')
#     now_date = pd.to_datetime(row['now_date'], errors='coerce')

#     if pd.isna(实付日期new) or pd.isna(应付日期):
#         return 0
    
#     if 实付日期new >= 应付日期:
#         return (实付日期new - 应付日期).days
    
#     if pd.isna(实付日期new) and now_date > 应付日期:
#         return (now_date - 应付日期).days
    
#     return 0

# # 应用函数并创建新的 '逾期天数' 列
# df_xz_concat['逾期天数'] = df_xz_concat.apply(calculate_overdue_days, axis=1)

# # 显示结果
# print(df_xz_concat['逾期天数'].value_counts())

In [356]:
df_xz_concat[df_xz_concat['逾期天数']>0]['rembursement_status'].value_counts()

rembursement_status
已逾期        32091
还款中         5141
续租中         4315
已买断         3983
已完成          771
未到首期还款日      170
检测中          126
Name: count, dtype: int64

# 逾期天数处理

In [357]:
def over_due_days(df):  
    # 转换日期为pandas的日期类型
    df['实付日期'] = pd.to_datetime(df['实付日期'])  
    df['应付日期'] = pd.to_datetime(df['应付日期'])  
      
    # 创建一个新的列来存储结果  
    df['逾期天数new'] = np.nan  # 初始化为NaN  
      
    # 应用逻辑判断  
    mask1 = df['状态'].isin(['已逾期', '未到还款日', '已取消','已提前支付'])  
    mask2 = df['实付日期'].isnull()  
    mask3 = (df['实付日期'] - df['应付日期']).dt.days <= 0  
      
    # 更新计算天数列  
    df.loc[mask1, '逾期天数new'] = df.loc[mask1, '逾期天数']  
    df.loc[~mask1 & mask2, '逾期天数new'] = 0   
    df.loc[~mask1 & ~mask2 & mask3, '逾期天数new'] = 0  
    df.loc[~mask1 & ~mask2 & ~mask3, '逾期天数new'] = (df['实付日期'] - df['应付日期']).dt.days  
      
    # 实付日期为空时设置计算天数为0：  
    # df.loc[df['实付日期'].isnull(), '逾期天数new'] = 0  
      
    return df 

    # df_ddfq = over_due_days(df_ddfq)

##  核查 

In [358]:
# df_ddfq.columns

In [359]:
# df_xz_concat.shape

In [360]:
# check_fq_df = df_xz_concat.merge(df_ddfq,on=['订单ID','当前期数'],how='left')
# check_fq_df.shape

In [361]:
# check_fq_df.columns

In [362]:
# check_fq_df[['订单ID', '当前应付租金_x', '实付金额','应付日期_x', '实付日期_x','status', '当前期数', 'is_relet','订单号_x','下单月份', '下单日期','还款状态', '实付金额new', '实付日期new', '逾期天数_x','状态_x'
            #  ,'当前应付租金_y', '实际支付金额', '应付日期_y', '实付日期_y', '状态_y', '逾期天数_y','逾期天数new']].to_excel('E:/myfile/业务相关数据/check_fq_df.xlsx')

# 确认是否到表现期

In [363]:
# import os
# file_name = os.path.basename(f_path_ck)  

# df_xz_concat['date_str'] = file_name.split('excel')[1].split('.')[0]

# df_xz_concat['date_str'].head(1)

In [364]:
# df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
# df_xz_concat['now_date']

In [365]:
# dfzh1 = over_due_days(dfzh1)

df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']).dt.date
# df_xz_concat['agr_days'] = (pd.to_datetime(df_xz_concat['now_date']) - df_xz_concat['应付日期']).dt.days 


In [366]:

from datetime import timedelta
df_xz_concat['agr_1d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=1)
df_xz_concat['agr_4d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=4)
df_xz_concat['agr_7d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=7)
df_xz_concat['agr_15d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=15)
df_xz_concat['agr_30d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=30)

df_xz_concat[['agr_30d','应付日期']].head(10)

,agr_30d,应付日期
0,2022-08-04,2022-07-05
1,2022-09-04,2022-08-05
2,2022-10-05,2022-09-05
3,2022-11-04,2022-10-05
4,2022-12-05,2022-11-05
5,2023-01-04,2022-12-05
6,2023-02-04,2023-01-05
7,2023-03-07,2023-02-05
8,2023-04-04,2023-03-05
9,2023-05-05,2023-04-05


In [367]:
import os

# file_name = os.path.basename(f_path_ck)  
df_xz_concat['date_str'] = df_xz_concat['now_date']
df_xz_concat['agr_1d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_1d) <= pd.to_datetime(df_xz_concat.date_str),1,0)

df_xz_concat['agr_4d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_4d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_7d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_7d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_15d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_15d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_30d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_30d) <= pd.to_datetime(df_xz_concat.date_str),1,0)


df_xz_concat[['agr_30d','应付日期','date_str','agr_30d_cum']].head(10)

,agr_30d,应付日期,date_str,agr_30d_cum
0,2022-08-04,2022-07-05,2024-12-05,1
1,2022-09-04,2022-08-05,2024-12-05,1
2,2022-10-05,2022-09-05,2024-12-05,1
3,2022-11-04,2022-10-05,2024-12-05,1
4,2022-12-05,2022-11-05,2024-12-05,1
5,2023-01-04,2022-12-05,2024-12-05,1
6,2023-02-04,2023-01-05,2024-12-05,1
7,2023-03-07,2023-02-05,2024-12-05,1
8,2023-04-04,2023-03-05,2024-12-05,1
9,2023-05-05,2023-04-05,2024-12-05,1


# 观察日定义  每月月底时间

In [368]:
from pandas.tseries.offsets import MonthEnd
def mob_date(df):
    df['下单日期'] = pd.to_datetime(df['下单日期'])
    df['实付日期new'] = pd.to_datetime(df['实付日期new'])
    for n in range(0,13):  
        df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1) 
        df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}'])
    return df

df_xz_concat = mob_date(df_xz_concat)
df_xz_concat[['下单日期','mob_date_0','mob_date_1','mob_date_2','mob_date_3','实付金额','paid_money_mob1','paid_date_mob12','paid_money_mob12']]

,下单日期,mob_date_0,mob_date_1,mob_date_2,mob_date_3,实付金额,paid_money_mob1,paid_date_mob12,paid_money_mob12
0,2022-07-05,2022-07-31,2022-08-31,2022-09-30,2022-10-31,357.95,357.95,2022-07-05,357.95
1,2022-07-05,2022-07-31,2022-08-31,2022-09-30,2022-10-31,357.95,357.95,2022-08-05,357.95
2,2022-07-05,2022-07-31,2022-08-31,2022-09-30,2022-10-31,357.95,NaN,2022-09-05,357.95
3,2022-07-05,2022-07-31,2022-08-31,2022-09-30,2022-10-31,357.95,NaN,2022-10-09,357.95
4,2022-07-05,2022-07-31,2022-08-31,2022-09-30,2022-10-31,357.95,NaN,2022-11-08,357.95
...,...,...,...,...,...,...,...,...,...
357685,2024-12-04,2024-12-31,2025-01-31,2025-02-28,2025-03-31,0.00,0.00,NaT,0.00
357686,2024-12-04,2024-12-31,2025-01-31,2025-02-28,2025-03-31,0.00,0.00,NaT,0.00
357687,2024-12-04,2024-12-31,2025-01-31,2025-02-28,2025-03-31,0.00,0.00,NaT,0.00
357688,2024-12-04,2024-12-31,2025-01-31,2025-02-28,2025-03-31,0.00,0.00,NaT,0.00


In [369]:
df_xz_concat['date_str']  = pd.to_datetime( df_xz_concat['date_str'] )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']) 

In [370]:
# df_xz_concat.columns

In [371]:
# df_xz_concat[df_xz_concat.order_id== 12514][['下单日期','mob_date_10','mob_date_11','实付金额','paid_money_mob10','paid_money_mob11','paid_date_mob12','paid_money_mob12']]


# 续租用户

In [372]:
df_xz_concat.is_relet.value_counts()

is_relet
0.0    324783
1.0     32757
2.0        62
Name: count, dtype: int64

In [373]:
df_relet = df_xz_concat[df_xz_concat.is_relet>=1]
df_relet.shape

(32819, 158)

## 总体订单口径每月当前逾期率

In [374]:
dfzh3.columns

Index(['id', 'order_id', 'order_number', 'order_time', 'search_time',
       'end_date', 'merchant_id', 'merchant_name', 'channel_id',
       'channel_name', 'model_number', 'specification', 'purchase_amount',
       'platform_package', 'all_deposit', 'all_rental', 'liquidated_damages',
       'buyout_balance_payment', 'advance_periods', 'advance_sum',
       'order_create_time', 'rembursement_status', 'paid_periods',
       'begin_overdue_time', 'overdue_periods', 'max_overdue_days',
       'now_overdue_days', 'total_receivable', 'contract_price',
       'rent_received', 'unearned_rent_revenue', 'paid_buyout_balance_payment',
       'total_received', 'total_unreceived', 'unrecovered_cost',
       'overdue_type', 'gross_profit_receivable', 'overdue_loss',
       'actual_gross_profit', 'overdue_total_unreceived', 'relet_periods',
       'relet_days', 'relet_rent_paid', 'activity_id', 'activity_source',
       'order_label', 'create_time', 'update_time', 'order_finish_date',
       'comm

In [375]:
dfzh3['是否二手'].drop_duplicates()

dfzh3['下单月份'] = dfzh3['下单日期'].dt.strftime('%Y-%m')

In [376]:
dfzh3["还款状态"] = dfzh3["rembursement_status"]

In [377]:
dfzh3_2 = dfzh3[dfzh3['是否二手']=='二手']
# dfzh3_2 = dfzh3_2[~dfzh3_2['model_number'].str.contains(r'电脑|游戏机|手表|修复仪')]
# dfzh3_2 = dfzh3_2[dfzh3_2["还款状态"]!='未到首期还款日']

In [378]:
dfzh3_2.shape

(2123, 83)

In [379]:
#总体每月当前逾期率 cpd1
df0 = pd.crosstab(dfzh3_2["下单月份"],dfzh3_2["还款状态"],margins=True)
df0["逾期率"]=df0["已逾期"]/df0[["已买断","已完成","已逾期","续租中","还款中","检测中"]].apply(lambda x:x.sum(),axis=1)
# df0["逾期率"]=df0["已逾期"]/df0[["已买断","已完成","续租中","检测中"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0["买断率"]=df0["已买断"]/df0[["已买断","已完成","续租中","检测中"]].apply(lambda x:x.sum(),axis=1)
df0["买断率"]=df0["买断率"].apply(lambda x:format(x,".2%"))
df0["归还率"]=(df0["已完成"]+df0["检测中"])/df0[["已买断","已完成","续租中","检测中"]].apply(lambda x:x.sum(),axis=1)
df0["归还率"]=df0["归还率"].apply(lambda x:format(x,".2%"))
df0["续租率"]=df0["续租中"]/df0[["已买断","已完成","续租中","检测中"]].apply(lambda x:x.sum(),axis=1)
df0["续租率"]=df0["续租率"].apply(lambda x:format(x,".2%"))
df0.to_excel('F:/需求/七月需求/二手手机数据.xlsx')#[["已买断","All","买断率"]]#[14:]

In [380]:
dfzh3["续租期数"] = dfzh3["relet_periods"] 
dfzh3["已还期数"] = dfzh3["paid_periods"] 


In [381]:
# 以下代码按顺序运行完到到期状态数据
# pd.crosstab(dfzh1["下单月份"],dfzh1["已还期数"],margins=True)
df_yq = dfzh3[dfzh3["还款状态"]=="已逾期"]

# df_yq=df[df["还款状态"]=="已逾期"]
# pd.crosstab(df_yq["下单月份"],df_yq["续租期数"],margins=True)

df_xz = dfzh3[dfzh3["续租期数"]>0]
# pd.crosstab(df_xz["下单月份"],df_xz["还款状态"],margins=True)

df_timeup = dfzh3[dfzh3["已还期数"]==12]


df_timeon = dfzh3[dfzh3["已还期数"]<12]
df_timeon_1 = pd.crosstab(df_timeon["下单月份"],df_timeon["还款状态"],margins=True).rename(columns={"已买断":"提前买断","已完成":"提前归还","已逾期":"租期逾期"})
df_timeup_xz = dfzh3[(dfzh3["已还期数"]==12)&(dfzh3["续租期数"]>0)]
df_timeup_xz_1 = pd.crosstab(df_timeup_xz["下单月份"],df_timeup_xz["还款状态"],margins=True).rename(columns={"已买断":"续租后买断","已完成":"续租后归还","已逾期":"续租后逾期","检测中":"续租归还检测中"})
df_timeup_nxz = dfzh3[(dfzh3["已还期数"]==12)&(dfzh3["续租期数"]<1)]
df_timeup_nxz_1 = pd.crosstab(df_timeup_nxz["下单月份"],df_timeup_nxz["还款状态"],margins=True).rename(columns={"已买断":"到期买断","已完成":"到期归还","已逾期":"到期逾期","检测中":"到期归还检测中","续租中":"待续租","还款中":"已付总租金待归还"})


In [382]:
#以及每个下单月份的订单在12期租期到期后到期买断、到期归还、到期逾期、到期续租的订单数，
#以及每个下单月份的订单在到期续租后买断、逾期、归还的订单数
# df_timeon_1.drop(columns=['检测中','续租中'],inplace = True)
df_due = pd.merge(pd.merge(df_timeon_1,df_timeup_nxz_1,left_on="下单月份",right_on="下单月份",how="left"),df_timeup_xz_1,left_on="下单月份",right_on="下单月份",how="left")
df_due=df_due.fillna(0)
df_due["合计"]=df_due["All_x"]+df_due["All_y"]+df_due["All"]
#df_due["合计"]=df_due["合计"].astype("int")
df_due.drop(columns=["All_x","All_y","All"],axis=1,inplace=True)
df_due=df_due.astype("int")
df_due#[["提前买断","到期买断","续租后买断"]]#[14:]#[14:]

还款状态,提前买断,提前归还,租期逾期,未到首期还款日,检测中,还款中,到期买断,到期归还,到期逾期,到期归还检测中,待续租,已付总租金待归还,续租后买断,续租后归还,续租后逾期,续租归还检测中,续租中,合计
下单月份,,,,,,,,,,,,,,,,,,
2022-07,0,1,8,0,0,0,6,1,0,0,0,0,3,0,0,0,3,22
2022-08,2,1,21,0,0,0,7,2,7,0,0,0,11,1,2,1,1,56
2022-09,4,2,10,0,0,0,4,2,5,0,0,0,9,2,5,0,1,44
2022-10,1,1,8,0,0,0,8,2,1,1,0,0,11,3,3,0,2,41
2022-11,5,0,8,0,0,0,6,6,1,0,0,0,10,1,4,0,2,43
2022-12,8,2,16,0,0,0,13,2,6,0,0,0,13,0,2,0,4,66
2023-01,17,4,64,0,0,0,42,16,14,3,0,0,45,0,13,0,3,221
2023-02,24,5,73,0,0,1,90,43,30,1,0,0,42,0,15,0,9,333
2023-03,24,13,85,0,0,0,71,32,25,3,0,0,41,0,12,0,12,318


In [383]:
# df_due['续租中']=df_due['续租中_x']+df_due['续租中_y']
# df_due.columns.tolist()

In [384]:
#以下为分别统计每个下单月的订单在12期租期内租期逾期、提前买断、提前归还的的订单占比，
#以及每个下单月份的订单在12期租期到期后到期买断、到期归还、到期逾期、到期续租的订单占比，
#以及每个下单月份的订单在到期续租后买断、逾期、归还的订单占比
#该代码需先运行上述代码方不会报错
# applymap(lambda x:format(x,".2%"))
# df_due.divide(df_due["合计"],axis=0).applymap(lambda x:format(x,".2%"))#[["提前买断","到期买断","续租后买断"]]
df_due=df_due.divide(df_due["合计"],axis=0).applymap(lambda x:format(x,".2%"))#[["提前买断","到期买断","续租后买断"]]

df_due

还款状态,提前买断,提前归还,租期逾期,未到首期还款日,检测中,还款中,到期买断,到期归还,到期逾期,到期归还检测中,待续租,已付总租金待归还,续租后买断,续租后归还,续租后逾期,续租归还检测中,续租中,合计
下单月份,,,,,,,,,,,,,,,,,,
2022-07,0.00%,4.55%,36.36%,0.00%,0.00%,0.00%,27.27%,4.55%,0.00%,0.00%,0.00%,0.00%,13.64%,0.00%,0.00%,0.00%,13.64%,100.00%
2022-08,3.57%,1.79%,37.50%,0.00%,0.00%,0.00%,12.50%,3.57%,12.50%,0.00%,0.00%,0.00%,19.64%,1.79%,3.57%,1.79%,1.79%,100.00%
2022-09,9.09%,4.55%,22.73%,0.00%,0.00%,0.00%,9.09%,4.55%,11.36%,0.00%,0.00%,0.00%,20.45%,4.55%,11.36%,0.00%,2.27%,100.00%
2022-10,2.44%,2.44%,19.51%,0.00%,0.00%,0.00%,19.51%,4.88%,2.44%,2.44%,0.00%,0.00%,26.83%,7.32%,7.32%,0.00%,4.88%,100.00%
2022-11,11.63%,0.00%,18.60%,0.00%,0.00%,0.00%,13.95%,13.95%,2.33%,0.00%,0.00%,0.00%,23.26%,2.33%,9.30%,0.00%,4.65%,100.00%
2022-12,12.12%,3.03%,24.24%,0.00%,0.00%,0.00%,19.70%,3.03%,9.09%,0.00%,0.00%,0.00%,19.70%,0.00%,3.03%,0.00%,6.06%,100.00%
2023-01,7.69%,1.81%,28.96%,0.00%,0.00%,0.00%,19.00%,7.24%,6.33%,1.36%,0.00%,0.00%,20.36%,0.00%,5.88%,0.00%,1.36%,100.00%
2023-02,7.21%,1.50%,21.92%,0.00%,0.00%,0.30%,27.03%,12.91%,9.01%,0.30%,0.00%,0.00%,12.61%,0.00%,4.50%,0.00%,2.70%,100.00%
2023-03,7.55%,4.09%,26.73%,0.00%,0.00%,0.00%,22.33%,10.06%,7.86%,0.94%,0.00%,0.00%,12.89%,0.00%,3.77%,0.00%,3.77%,100.00%


In [385]:
# 以下为每月预收租金期数占比
dfzh3["预收期数"] = dfzh3["advance_periods"] 
df_ys=pd.crosstab(dfzh3["下单月份"],dfzh3["预收期数"],margins=True)
df_ys.divide(df_ys["All"],axis=0).applymap(lambda x:format(x,".2%"))


预收期数,0,1,2,3,4,5,6,7,8,9,10,12,All
下单月份,,,,,,,,,,,,,
2022-07,0.00%,13.64%,36.36%,45.45%,4.55%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2022-08,0.00%,1.79%,28.57%,48.21%,12.50%,5.36%,3.57%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2022-09,0.00%,0.00%,34.09%,45.45%,18.18%,2.27%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2022-10,2.44%,24.39%,41.46%,14.63%,12.20%,4.88%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2022-11,0.00%,11.63%,51.16%,16.28%,18.60%,2.33%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2022-12,0.00%,18.18%,42.42%,19.70%,18.18%,1.52%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2023-01,0.45%,16.29%,54.30%,19.00%,5.88%,2.26%,0.45%,0.45%,0.00%,0.00%,0.00%,0.90%,100.00%
2023-02,0.00%,5.71%,60.96%,17.42%,8.41%,4.80%,1.80%,0.00%,0.30%,0.30%,0.00%,0.30%,100.00%
2023-03,0.63%,15.41%,59.43%,11.01%,4.40%,3.14%,5.66%,0.00%,0.31%,0.00%,0.00%,0.00%,100.00%


## 平台套餐统计

In [386]:
dfzh3['平台套餐'] = dfzh3['platform_package']
dfzh3['租赁方案'] = np.where(dfzh3.平台套餐=='租完即送','租完即送','租完归还/可买断')
dfzh3['租赁方案'].value_counts()

租赁方案
租完归还/可买断    24828
租完即送         2249
Name: count, dtype: int64

In [387]:

zl_class_df = pd.crosstab(dfzh3['下单月份'],dfzh3['租赁方案'],margins=True)
zl_class_df.divide(zl_class_df["All"],axis=0).applymap(lambda x:format(x,".2%"))


租赁方案,租完即送,租完归还/可买断,All
下单月份,,,
2022-07,27.27%,72.73%,100.00%
2022-08,16.07%,83.93%,100.00%
2022-09,2.27%,97.73%,100.00%
2022-10,0.00%,100.00%,100.00%
2022-11,6.98%,93.02%,100.00%
2022-12,12.12%,87.88%,100.00%
2023-01,19.46%,80.54%,100.00%
2023-02,9.01%,90.99%,100.00%
2023-03,14.78%,85.22%,100.00%


In [388]:
# df_yq_fq.columns[:100]

## 每月各渠道逾期率 sheet 数据输出

In [389]:
# df_fq_concat.describe().to_excel('E:/myfile/业务相关数据/df_desc.xlsx')

In [390]:
# df_yq_fq = df_yq_fq.rename(columns={'1,0':'1','2.0':'2','3.0':'3','4.0':'4','5.0':'5','6.0':'6','7.0':'7','8.0':'8','9.0':'9','10.0':'10','11.0':'11','12.0':'12'})
# df_yq_fq.columns

# 免审 fstq 表现

In [391]:
sql = '''-- 免人审 
SELECT date(r.time) as risk_date 
,replace(case when JSON_VALID(r.data) THEN JSON_EXTRACT(r.data, '$.grade_msg') end,'"','') as grade_msg 
,replace(case when JSON_VALID(r.data) THEN JSON_EXTRACT(r.data, '$.result') end,'"','') as risk_result 
,replace(case when JSON_VALID(r.data) THEN JSON_EXTRACT(r.data, '$.rejected') end,'"','') as reject_tag 
,replace(case when JSON_VALID(r.data) THEN JSON_EXTRACT(r.data, '$.trace_id') end,'"','') as trace_id 
,r.mobile,r.id_card,r.data  
from db_credit.risk r  
where  r.data like '%免人审%'
# and  r.time > '2024-07-01 00:00:00'

;
'''
risk_data = query(sql)
risk_data.shape

(46473, 8)

In [392]:
sql = '''-- 免人审 
select oa.* 
from db_credit.order_association oa 

;
'''
order_data = query(sql)
order_data.shape

(1424690, 5)

In [393]:
risk_data.sort_values(by=["trace_id","risk_date"],inplace=True)
risk_data.drop_duplicates(subset=["trace_id"],keep="last",inplace=True)
risk_data.shape

(46473, 8)

In [394]:
ms_df = risk_data.merge(order_data,left_on='trace_id',right_on='risk_trace_id',how='left')
ms_df.shape

(46566, 13)

In [395]:
ms_df.drop_duplicates(subset=["order_id"],keep="last",inplace=True)
ms_df.shape

(45850, 13)